In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/positive-and-negative-test-cases/Labelled_Test_Cases.csv


In [2]:
df = pd.read_csv('/kaggle/input/positive-and-negative-test-cases/Labelled_Test_Cases.csv', encoding='latin1')

In [3]:
df.shape

(3000, 5)

In [4]:
df.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Neg,A customer cannot stream a movie if they purc...,NaN,NaN,NaN
1,Neg,A database management system such as Oracle D...,NaN,NaN,NaN
2,Neg,All credit card information will be not be se...,NaN,NaN,NaN
3,Neg,Not all movies shall be streamed on demand at ...,NaN,NaN,NaN
4,Neg,An employee is unable to successfully use the...,NaN,NaN,NaN
5,Neg,Any interface between a user and the automate...,NaN,NaN,NaN
6,Neg,"Table constraints like a Primary key, Foreign ...",NaN,NaN,NaN
7,Neg,File selection dialog does not show supported ...,NaN,NaN,NaN
8,Neg,Field length shown to the user on the page and...,NaN,NaN,NaN
9,Neg,Cancel button functionality is not working in ...,NaN,NaN,NaN


In [5]:
column_to_keep = ['v1','v2']
df = df[column_to_keep]

In [6]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [7]:
df.v1.value_counts()

v1
Pos    2100
Neg     900
Name: count, dtype: int64

In [8]:
# Import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
import pandas as pd

# Clean and rename columns
df = df[['v1', 'v2']].dropna()  # Keep only the relevant columns and drop rows with NaN values
df.columns = ['label', 'text']  # Rename columns for consistency

# Map labels to numerical values
df['label'] = df['label'].map({'Neg': 0, 'Pos': 1})

# Define classifiers
classifiers = {
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'Logistic Regression': LogisticRegression(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Iterate through classifiers
for name, clf in classifiers.items():
    # Create a pipeline for text feature extraction and classification
    if name == 'XGBoost':  # XGBoost doesn't require CountVectorizer and TfidfTransformer
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(df['text'])
        y = df['label']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
    else:
        model = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('transformer', TfidfTransformer()),
            ('classifier', clf)
        ])
        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.3, random_state=42)

        # Train the model
        model.fit(X_train, y_train)

        # Make predictions on the test set
        predictions = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)

    # Print results
    print(f'\nResults for {name}:')
    print(f'Accuracy: {accuracy}')
    print(f'Classification Report:\n{report}')


Results for Multinomial Naive Bayes:
Accuracy: 0.8
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.43      0.58       288
           1       0.78      0.97      0.87       612

    accuracy                           0.80       900
   macro avg       0.83      0.70      0.72       900
weighted avg       0.82      0.80      0.78       900


Results for Decision Tree:
Accuracy: 0.9233333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.84      0.88       288
           1       0.93      0.96      0.94       612

    accuracy                           0.92       900
   macro avg       0.92      0.90      0.91       900
weighted avg       0.92      0.92      0.92       900


Results for Random Forest:
Accuracy: 0.8933333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.73      0.81       288
  